# Предсказание победителя в игре Dota 2

Ссылка на Kaggle соревнование: [Dota 2: Win Probability Prediction](https://kaggle.com/join/coursera_ml_dota2_contest)

## Предметная область: Игра Dota 2

[Dota 2](https://ru.wikipedia.org/wiki/Dota_2) — многопользовательская компьютерная игра жанра [MOBA](https://ru.wikipedia.org/wiki/MOBA). Игроки играют между собой матчи. В каждом матче участвует две команды, 5 человек в каждой. Одна команда играет за светлую сторону (The Radiant), другая — за тёмную (The Dire). Цель каждой команды — уничтожить главное здание базы противника (трон).

Существуют [разные режимы игры](http://dota2.gamepedia.com/Game_modes/ru), мы будем рассматривать режим [Captain's Mode](http://dota2.gamepedia.com/Game_modes/ru#Captain.27s_Mode), в формате которого происходит большая часть киберспортивных мероприятий по Dota 2.

### Как проходит матч

#### 1. Игроки выбирают героев

Всего в игре чуть более 100 различных героев (персонажей). В начале игры, команды в определенном порядке выбирают героев себе и запрещают выбирать определенных героев противнику (баны). Каждый игрок будет управлять одним героем, в рамках одного матча не может быть несколько одинаковых героев.  Герои различаются между собой своими характеристиками и способностями. От комбинации выбранных героев во многом зависит успех команды.

![](http://imgur.com/XFr4HYE.jpg)

#### 2. Основная часть

Игроки могут получать золото и опыт за убийство чужих героев или прочих юнитов. Накопленный опыт влияет на уровень героя, который в свою очередь позволяет улучшать способности. За накопленное золото игроки покупают предметы, которые улучшают характеристики героев или дают им новые способности.

После смерти герой отправляется в "таверну" и возрождается только по прошествии некоторого времени, таким образом команда на некоторое время теряет игрока, однако игрок может досрочно выкупить героя из таверны за определенную сумму золота.

В течение игры команды развивают своих героев, обороняют свою часть поля и нападают на вражескую.

![](http://imgur.com/5b0SlQb.jpg)

#### 3. Конец игры

Игра заканчивается, когда одна из команд разрушет определенное число "башен" противника и уничтожает трон.

![](http://imgur.com/Du79Kzf.jpg)

## Задача: предсказание победы по данным о первых 5 минутах игры

По первым 5 минутам игры предсказать, какая из команд победит: Radiant или Dire?

## Набор данных
Признаки, представленные в таблице `dota_features.csv`, по мнению экспертов в предметной области являются наиболее важными для решения задачи предсказания победы команды.

#### Описание признаков в таблице

- `match_id`: идентификатор матча в наборе данных
- `start_time`: время начала матча (unixtime)
- `lobby_type`: тип комнаты, в которой собираются игроки (расшифровка в `dictionaries/lobbies.csv`)
- Наборы признаков для каждого игрока (игроки команды Radiant — префикс `rN`, Dire — `dN`):
    - `r1_hero`: герой игрока (расшифровка в dictionaries/heroes.csv)
    - `r1_level`: максимальный достигнутый уровень героя (за первые 5 игровых минут)
    - `r1_xp`: максимальный полученный опыт
    - `r1_gold`: достигнутая ценность героя
    - `r1_lh`: число убитых юнитов
    - `r1_kills`: число убитых игроков
    - `r1_deaths`: число смертей героя
    - `r1_items`: число купленных предметов
- Признаки события "первая кровь" (first blood). Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение
    - `first_blood_time`: игровое время первой крови
    - `first_blood_team`: команда, совершившая первую кровь (0 — Radiant, 1 — Dire)
    - `first_blood_player1`: игрок, причастный к событию
    - `first_blood_player2`: второй игрок, причастный к событию
- Признаки для каждой команды (префиксы `radiant_` и `dire_`)
    - `radiant_bottle_time`: время первого приобретения командой предмета "bottle"
    - `radiant_courier_time`: время приобретения предмета "courier" 
    - `radiant_flying_courier_time`: время приобретения предмета "flying_courier" 
    - `radiant_tpscroll_count`: число предметов "tpscroll" за первые 5 минут
    - `radiant_boots_count`: число предметов "boots"
    - `radiant_ward_observer_count`: число предметов "ward_observer"
    - `radiant_ward_sentry_count`: число предметов "ward_sentry"
    - `radiant_first_ward_time`: время установки командой первого "наблюдателя", т.е. предмета, который позволяет видеть часть игрового поля
- Итог матча (данные поля отсутствуют в тестовой выборке, поскольку содержат информацию, выходящую за пределы первых 5 минут матча)
    - `duration`: длительность
    - `radiant_win`: 1, если победила команда Radiant, 0 — иначе
    - Состояние башен и барраков к концу матча (см. описание полей набора данных)
        - `tower_status_radiant`
        - `tower_status_dire`
        - `barracks_status_radiant`
        - `barracks_status_dire`

**Целевой признак** содержится в признаке radiant_win: 1, если победила команда Radiant, 0 — если победила команда Dire. 

## Метрика качества

В качестве метрики качества мы будем использовать площадь под ROC-кривой (AUC-ROC).

## Предобработка данных

Импортируем библиотеки и датасет:

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('dota_features.csv', index_col='match_id').reset_index(drop = True)
df

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97225,1450265551,1,47,4,1706,1198,17,0,1,8,...,4,3,0,-24.0,2032,0,1792,1975,48,63
97226,1450277704,0,43,4,1793,1416,17,0,1,5,...,3,2,0,-17.0,1734,1,2038,6,63,3
97227,1450291848,1,98,4,1399,540,1,0,0,5,...,1,3,1,-15.0,2906,0,1796,1846,51,63
97228,1450292986,1,100,3,1135,766,6,0,2,6,...,3,3,1,-42.0,951,0,2039,2047,63,63


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97230 entries, 0 to 97229
Columns: 108 entries, start_time to barracks_status_dire
dtypes: float64(12), int64(96)
memory usage: 80.1 MB


Все данные в числовом формате.

Сохраним целевой признак в переменную y:

In [ ]:
y = df.radiant_win
y.shape

(97230,)

В переменную features сохраним признаки, кроме целевого признака и еще нескольких столбцов, которые содержат информацию, связанную с итогами матча, а также времени начала матча и типа комнаты (эти признаки, очевидно, не несут в себе информации, полезной для обучения нашей модели):
- lobby_type
- start_time
- duration
- tower_status_radiant
- tower_status_dire
- barracks_status_radiant
- barracks_status_dire

In [ ]:
features = df.drop(['radiant_win', 'lobby_type', 'start_time', 'duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'], axis = 1)
features

,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_level,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,11,5,2098,1489,20,0,0,7,67,3,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,42,4,1188,1033,9,0,1,12,49,4,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,33,4,1319,1270,22,0,0,12,98,3,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,29,4,1779,1056,14,0,0,5,30,2,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,13,4,1431,1090,8,1,0,8,27,2,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97225,47,4,1706,1198,17,0,1,8,7,2,...,0,-29.0,180.0,-76.0,180.0,3,4,3,0,-24.0
97226,43,4,1793,1416,17,0,1,5,26,3,...,0,-5.0,NaN,-82.0,NaN,4,3,2,0,-17.0
97227,98,4,1399,540,1,0,0,5,11,4,...,2,-32.0,249.0,-70.0,NaN,1,1,3,1,-15.0
97228,100,3,1135,766,6,0,2,6,72,5,...,0,-21.0,254.0,-85.0,183.0,5,3,3,1,-42.0


Посмотрим, в каких столбцах есть пропуски:

In [ ]:
features.isna().sum().sort_values(ascending = False).head(15)

first_blood_player2            43987
radiant_flying_courier_time    27479
dire_flying_courier_time       26098
first_blood_time               19553
first_blood_team               19553
first_blood_player1            19553
dire_bottle_time               16143
radiant_bottle_time            15691
radiant_first_ward_time         1836
dire_first_ward_time            1826
radiant_courier_time             692
dire_courier_time                676
r5_kills                           0
r4_items                           0
r3_items                           0
dtype: int64

Из описания признаков мы знаем, что если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение. Значит, признаки в столбцах first_blood_time,
first_blood_team, first_blood_player1, first_blood_player2 можно заменить на нули.

Признаки radiant_flying_courier_time и dire_flying_courier_time обозначают время приобретения предмета "flying_courier". Вероятно, пропуск означает, что этот предмет не был приобретен данной командой в ходе игры. Скорее всего, пропуски в этих столбцах тоже можно заменить нулями.

То же самое относится и к другим признакам с пропусками:
- dire_bottle_time               
- radiant_bottle_time            
- radiant_first_ward_time         
- dire_first_ward_time            
- radiant_courier_time             
- dire_courier_time                

In [ ]:
features = features.fillna(0)
features.isna().sum().sum()

0

Импортируем библиотеки для предобработки данных и обучения моделей и метрики качества:

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

import time
import datetime

## Градиентный бустинг

Попробуем обучить модель градиентного бустинга, высчитаем метрику качества на кросс-валидации и посмотрим, как она меняется с увеличением количества деревьев:

In [ ]:
start_time = datetime.datetime.now()

time.sleep(3)

scores = []
folds = KFold(n_splits = 5, shuffle = True, random_state = 1)
for k in range(10, 51, 10):
    gbc = GradientBoostingClassifier(random_state = 1, n_estimators = k, learning_rate = 0.5)
    score = cross_val_score(gbc, features, y, cv = folds, scoring = 'roc_auc')
    scores.append(score)
display(scores)

for i in scores:
    print(round(np.mean(i), 4))

print ('Time elapsed:', datetime.datetime.now() - start_time)

[array([0.68560771, 0.68684191, 0.68938608, 0.67923921, 0.68516071]),
 array([0.6954447 , 0.69994919, 0.70172171, 0.69186507, 0.69625441]),
 array([0.70087222, 0.70505842, 0.7071958 , 0.69829634, 0.70112137]),
 array([0.70438323, 0.70817775, 0.71030125, 0.70190069, 0.7039526 ]),
 array([0.70621643, 0.7111071 , 0.71325159, 0.70443683, 0.70590954])]

0.6852
0.697
0.7025
0.7057
0.7082
Time elapsed: 0:17:00.303590


Как видим, качество модели повышается с увеличением количества деревьев. При этом модель обучается слишком долго.

## Логистическая регрессия

Логистическая регрессия должна обучаться быстрее, чем градиентный бустинг. Попробуем обучить несколько моделей с разными параметрами регуляризации и сравним их качество на кросс-валидации.

Масштабируем признаки:

In [ ]:
scal = StandardScaler()
scal.fit(features)
X0 = pd.DataFrame(scal.transform(features), columns = features.columns)

In [ ]:
start_time = datetime.datetime.now()

time.sleep(3)

scores0 = []
folds = KFold(n_splits = 5, shuffle = True, random_state = 1)
for k in [0.001, 0.01, 0.1, 1.0]:
    logreg = LogisticRegression(random_state = 1, C = k)
    score = cross_val_score(logreg, X0, y, cv = folds, scoring = 'roc_auc')
    scores0.append(score)
display(scores0)

for i in scores0:
    print('Средний AUC-ROC:', round(np.mean(i), 4))

print ('Time elapsed:', datetime.datetime.now() - start_time)

[array([0.71129324, 0.72005823, 0.71730233, 0.71456213, 0.71740981]),
 array([0.7114275 , 0.72042863, 0.71760376, 0.71499492, 0.71707668]),
 array([0.71139104, 0.72041421, 0.71760655, 0.71501943, 0.71696641]),
 array([0.71138501, 0.72041166, 0.7176097 , 0.71502025, 0.71695207])]

Средний AUC-ROC: 0.7161
Средний AUC-ROC: 0.7163
Средний AUC-ROC: 0.7163
Средний AUC-ROC: 0.7163
Time elapsed: 0:00:27.006514


Результаты сопоставимы с результатами модели градиентного бустинга, но модель обучается значительно быстрее.

Теперь попробуем обучить модель, исключив закодированные категориальные признаки с героями игры:

In [ ]:
features_num = list(features.columns)
print(len(features_num))

features_cat = ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']

for i in features_cat:
    features_num.remove(i)

len(features_num)

100


90

In [ ]:
scaler = StandardScaler()
scaler.fit(features[features_num])
X1 = pd.DataFrame(scaler.transform(features[features_num]), columns = features_num)

In [ ]:
start_time = datetime.datetime.now()

time.sleep(3)

scores1 = []
folds = KFold(n_splits = 5, shuffle = True, random_state = 1)
for k in [0.001, 0.01, 0.1, 1.0]:
    logreg = LogisticRegression(random_state = 1, C = k)
    score = cross_val_score(logreg, X1, y, cv = folds, scoring = 'roc_auc')
    scores1.append(score)
display(scores1)

for i in scores1:
    print('Средний AUC-ROC:', round(np.mean(i), 4))

print ('Time elapsed:', datetime.datetime.now() - start_time)

[array([0.71141743, 0.72017674, 0.71721692, 0.7144635 , 0.71743645]),
 array([0.71154013, 0.72057888, 0.71751394, 0.71489546, 0.71712114]),
 array([0.71150866, 0.72058126, 0.71751891, 0.71492553, 0.71700583]),
 array([0.71150434, 0.72057956, 0.71751908, 0.71492824, 0.71699432])]

Средний AUC-ROC: 0.7161
Средний AUC-ROC: 0.7163
Средний AUC-ROC: 0.7163
Средний AUC-ROC: 0.7163
Time elapsed: 0:00:26.437634


Качество улучшилось незначительно. Наилучший результат показывает модель с параметром регуляризации 0.01.

Мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясним из данных, сколько различных идентификаторов героев существует в данной игре:

In [ ]:
for i in features_cat:
    print(i, '-', features[i].nunique())

r1_hero - 108
r2_hero - 108
r3_hero - 108
r4_hero - 108
r5_hero - 108
d1_hero - 108
d2_hero - 108
d3_hero - 108
d4_hero - 108
d5_hero - 108


Сохраним все уникальные номера в переменной hero_ids:

In [ ]:
import collections
hero_ids = collections.Counter()

for i in features_cat:
    for h in features[i].unique():
        if hero_ids[h] not in hero_ids:
            hero_ids[h] = 1
        else:
            hero_ids[h] += 1

hero_ids = list(sorted(hero_ids.keys()))
print(hero_ids)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 109, 110, 112]


Всего в игре имеется 108 различных героев. Сформируем 108 признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire.

In [ ]:
r_heroes = ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero']
d_heroes = ['d1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']

In [ ]:
X_pick = pd.DataFrame(np.zeros((features.shape[0], 108)), columns = hero_ids)

for i in features.index:
    for r in r_heroes:
        X_pick.loc[i, features.loc[i, r]] = 1
    for d in d_heroes:
        X_pick.loc[i, features.loc[i, d]] = -1

Добавим полученные признаки к числовым:

In [ ]:
X1.shape

(97230, 90)

In [ ]:
X2 = pd.concat([X1, X_pick], axis = 1)

In [ ]:
X2.shape

(97230, 198)

Обучим модель логистической регрессии с новыми признаками:

In [ ]:
start_time = datetime.datetime.now()

time.sleep(3)

scores2 = []
folds = KFold(n_splits = 5, shuffle = True, random_state = 1)
logreg = LogisticRegression(random_state = 1, C = 0.1, max_iter = 1000)
score = cross_val_score(logreg, X2, y, cv = folds, scoring = 'roc_auc')
scores2.append(score)
display(scores2)

print('Средний AUC-ROC:', round(np.mean(scores2), 4))
print ('Time elapsed:', datetime.datetime.now() - start_time)

[array([0.75115989, 0.75467943, 0.75105004, 0.75036993, 0.75155796])]

Средний AUC-ROC: 0.7518
Time elapsed: 0:00:25.547235


Качество модели значительно улучшилось. Алгоритм смог обнаружить закономерности в данных: некоторые герои выигрывают чаще, чем другие.

## Проверка модели на тестовой выборке

Загрузим тестовую выборку:

In [ ]:
test = pd.read_csv('dota_features_test.csv')
test

,match_id,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,6,1430287923,0,93,4,1103,1089,8,0,1,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
1,7,1430293357,1,20,2,556,570,1,0,0,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
2,10,1430301774,1,112,2,751,808,1,0,0,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
3,13,1430323933,1,27,3,708,903,1,1,1,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
4,16,1430331112,1,39,4,1259,661,4,0,0,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17172,114369,1450212780,7,11,5,2054,1941,27,0,1,...,1,8.0,253.0,-87.0,NaN,4,3,2,1,-33.0
17173,114377,1450222875,1,3,3,748,605,1,0,0,...,1,-1.0,133.0,-85.0,184.0,2,3,4,1,-18.0
17174,114378,1450223593,1,85,2,575,499,0,0,0,...,0,20.0,133.0,-88.0,239.0,4,4,4,0,-36.0
17175,114393,1450244771,0,7,4,1844,1176,8,1,2,...,0,-28.0,NaN,-83.0,NaN,1,4,1,0,NaN


Сохраним номера матчей:

In [ ]:
ids = test.match_id

Проведем предобработку:

In [ ]:
test = test.drop(['match_id', 'start_time', 'lobby_type'], axis = 1)
test

,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_level,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,93,4,1103,1089,8,0,1,9,102,3,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
1,20,2,556,570,1,0,0,9,6,4,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
2,112,2,751,808,1,0,0,13,26,2,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
3,27,3,708,903,1,1,1,11,91,2,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
4,39,4,1259,661,4,0,0,9,93,5,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17172,11,5,2054,1941,27,0,1,8,28,4,...,1,8.0,253.0,-87.0,NaN,4,3,2,1,-33.0
17173,3,3,748,605,1,0,0,12,22,3,...,1,-1.0,133.0,-85.0,184.0,2,3,4,1,-18.0
17174,85,2,575,499,0,0,0,8,102,3,...,0,20.0,133.0,-88.0,239.0,4,4,4,0,-36.0
17175,7,4,1844,1176,8,1,2,8,32,3,...,0,-28.0,NaN,-83.0,NaN,1,4,1,0,NaN


In [ ]:
test = test.fillna(0)

In [ ]:
test_num = pd.DataFrame(scaler.transform(test[features_num]), columns = features_num)

In [ ]:
test_cat = pd.DataFrame(np.zeros((test.shape[0], 108)), columns = hero_ids)

for i in test.index:
    for r in r_heroes:
        test_cat.loc[i, test.loc[i, r]] = 1
    for d in d_heroes:
        test_cat.loc[i, test.loc[i, d]] = -1

In [ ]:
X_test = pd.concat([test_num, test_cat], axis = 1)

In [ ]:
X_test = X_test.fillna(0)

Обучим модель и сохраним предсказания вероятностей победы команды Radiant на основе данных тестовой выборки:

In [ ]:
model = LogisticRegression(random_state = 1, C = 0.1, max_iter = 1000)
model.fit(X2, y)
pred = pd.Series(model.predict_proba(X_test)[:, 1])

In [ ]:
final = pd.concat([ids, pred], axis = 1)
final.columns = ['match_id', 'radiant_win']
final

,match_id,radiant_win
0,6,0.835394
1,7,0.773838
2,10,0.202266
3,13,0.870120
4,16,0.261925
...,...,...
17172,114369,0.712144
17173,114377,0.625001
17174,114378,0.228760
17175,114393,0.615640


In [ ]:
print('Минимальное значение прогноза:', round(final.radiant_win.min(), 4))
print('Максимальное значение прогноза:', round(final.radiant_win.max(), 4))

Минимальное значение прогноза: 0.0083
Максимальное значение прогноза: 0.9965


In [ ]:
#final.to_csv('dota_final.csv')

К сожалению, узнать результат на тестовой выборке не представляется возможным, поскольку отправить решение на Kaggle невозможно. :(

## Выводы

- Модель градиентного бустинга обучается слишком долго на наших данных. Качество модели улучшается с увеличением количества деревьев, но чем больше деревьев - тем больше времени занимает кросс-валидация и обучение модели. Качество модели с 50 деревьями - около 0.7082, процесс занимает более 16 минут.


- Логистическая регрессия обучается в разы быстрее. Модель, обученная на начальном наборе признаков, показывает результат, сопоставимый с результатом модели градиентного бустинга. Лучшее качество на кросс-валидации - 0.7163 (при параметре регуляризации 0.01). Процесс занимает около 30 секунд.


- При исключении закодированных категориальных признаков качество модели почти не изменилось. Мы передаем модели числа, каждым из которых закодирован определенный герой игры. Но алгоритм воспринимает эти числа как градацию одного признака. В результате алгоритм не обнаружил закономерности в этих данных и практически не учитывал их при обучении модели.


- С использованием новых признаков (108 признаков - по количеству героев в игре) качество модели значительно улучшилось. Теперь алгоритм смог обнаружить закономерность в данных: некоторые герои выигрывают чаще, чем другие. Новые признаки были использованы при обучении модели и улучшили ее качество. Средний AUC-ROC: 0.7518, обучение модели длится менее 30 секунд.